In [1]:
import datetime as dt
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import statsmodels.formula.api as smf

import scipy.stats as stats

sns.set_theme(style="whitegrid")

## Load Landings Records


In [2]:
landings = pd.read_csv("../data/derived/landings_cleaned.csv", low_memory=False)

In [3]:
landings = landings[landings.species_en == "Greenland halibut"]

In [4]:
landings["is_ice"] = landings.vessel_type.map({"ice": 1, "water": 0})


def weighted_mean(df):
    return (df.is_ice * df.value).sum() / df.value.sum()


ice_proportion = landings.groupby(["seasonal_year", "seller_id"]).apply(
    weighted_mean, include_groups=False
)

In [5]:
fisher_type = ice_proportion.rename("ice_proportion_of_income").reset_index()


def determine_type(proportion):
    if proportion == 0:
        return "water-only"
    if proportion == 1:
        return "ice-only"
    if proportion >= 0.5:
        return "majority-ice"
    return "majority-water"


fisher_type["type"] = fisher_type.ice_proportion_of_income.apply(determine_type)

In [6]:
fisher_type.to_csv("../data/derived/fishertypes.csv", index=None)